In [94]:
import numpy as np
import os.path as op
import pickle as pkl
import pandas as pd
import csv
import os

## Create dictionary from behavioural csv(s)

### for train set

In [95]:
csv_file = "/Users/nalsharif/science/PING/train_img_beh.csv"

delimiter = ','
with open(csv_file, 'r') as data_file:
    data = csv.reader(data_file, delimiter=delimiter)
    headers = next(data)[1:] # header names starting from 2nd column in csv
    behav = dict()
    for row in data:
        behav[  row[0]  ] = dict()
        for idx, item in enumerate(row[1:]):
            try:
                behav[row[0]][  headers[idx]  ] = float(item)
            except ValueError as e:
                behav[row[0]][  headers[idx]  ] = float('nan')
                
# behav

### FUTURE REF: for test set

do the above + following again for the test set, just change csv but keep other variables, change pkl names

```
csv_file = "/Users/nalsharif/science/PING/test_img_beh.csv"

delimiter = ','
with open(csv_file, 'r') as data_file:
    data = csv.reader(data_file, delimiter=delimiter)
    headers = next(data)[1:] # header names starting from 2nd column in csv
    test_behav = dict()
    for row in data:
        test_behav[  row[0]  ] = dict()
        for idx, item in enumerate(row[1:]):
            try:
                test_behav[row[0]][  headers[idx]  ] = float(item)
            except ValueError as e:
                test_behav[row[0]][  headers[idx]  ] = float('nan')
```
                

### Get pickles from output dir

In [96]:
datadir = '/Users/nalsharif/science/PING/PING_bids/desikan/'
pickles = [op.join(datadir, fl) for fl in os.listdir(datadir) if fl.endswith('.pkl')]
pickles

['/Users/nalsharif/science/PING/PING_bids/desikan/desikan_locality_statistic.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_number_non_zeros.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_study_mean_connectome.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_degree_distribution.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_betweenness_centrality.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_edge_weight.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_eigen_sequence.pkl',
 '/Users/nalsharif/science/PING/PING_bids/desikan/desikan_clustering_coefficients.pkl']

In [97]:
local_stat = pkl.load(open(pickles[0], 'rb'))
locstat = local_stat['locality_statistic']

numb_nonzero = pkl.load(open(pickles[1], 'rb'))
nnonzero = numb_nonzero['number_non_zeros']

study_mean_conn = pkl.load(open(pickles[2], 'rb'))
meanconn = study_mean_conn['study_mean_connectome']

degree_dist = pkl.load(open(pickles[3], 'rb'))
degdist = degree_dist['degree_distribution']

btwn_central = pkl.load(open(pickles[4], 'rb'))
bwcent = btwn_central['betweenness_centrality']

edge_weight = pkl.load(open(pickles[5], 'rb'))
edgewt = edge_weight['edge_weight']

eigen_seq = pkl.load(open(pickles[6], 'rb'))
eigseq = eigen_seq['eigen_sequence']

coef_clust = pkl.load(open(pickles[7], 'rb'))
cfclust = coef_clust['clustering_coefficients']

these are organised with subjs as keys: locstat, nnonzero, bwcent, edgewt, eigseq, cfclust

these are organised with other feat as keys: degdist

these are organised as an array: meanconn


### Replace subjids in dictionaries

In [98]:
locstat_new = {}
for sub in locstat.keys():
    locstat_new[sub.split("-")[1].split("_")[0]] = locstat[sub]
    
nnonzero_new = {}
for sub in nnonzero.keys():
    nnonzero_new[sub.split("-")[1].split("_")[0]] = nnonzero[sub]
    
bwcent_new = {}
for sub in bwcent.keys():
    bwcent_new[sub.split("-")[1].split("_")[0]] = bwcent[sub]
    
edgewt_new = {}
for sub in edgewt.keys():
    edgewt_new[sub.split("-")[1].split("_")[0]] = edgewt[sub]
    
eigseq_new = {}
for sub in eigseq.keys():
    eigseq_new[sub.split("-")[1].split("_")[0]] = eigseq[sub]
    
cfclust_new = {}
for sub in cfclust.keys():
    cfclust_new[sub.split("-")[1].split("_")[0]] = cfclust[sub]
    
degdist_new = {}
for degtype in degdist.keys():
    degdist_new[degtype] = {}
    for sub in degdist[degtype].keys():
        degdist_new[degtype][sub.split("-")[1].split("_")[0]] = degdist[degtype][sub]

maybe use this code to create dataframes of everything at the same time?
- degseq_df = pd.DataFrame.from_dict(sub,orient = 'index')

### Pull list of subjids from both ndmg data + behav dict

In [99]:
pkl_subjids = []
for item in locstat.keys():
    pkl_subjids.append(item.split("-")[1].split("_")[0])
    subjids = [subj for subj in pkl_subjids if subj in behav]
    
subjids = list(set(subjids))

In [100]:
len(subjids), type(subjids)

(597, list)

### Make dictionary of just ages (for future reference/binning purposes, etc)

In [101]:
age = {}
for subj in subjids:
    try:
        age[subj] = behav[subj]['Age']
    except KeyError as e:
        age[subj] = 0

age['P0007']

16.42

### Save pickles for future use

In [102]:
# dat connectome feature info
pkl.dump(locstat_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_locstat.pkl','wb'))
pkl.dump(nnonzero_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_nnonzero.pkl','wb'))
pkl.dump(bwcent_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_bwcent.pkl','wb'))
pkl.dump(edgewt_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_edgewt_new.pkl','wb'))
pkl.dump(eigseq_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_eigseq.pkl','wb'))
pkl.dump(cfclust_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_cfclust.pkl','wb'))
pkl.dump(degdist_new, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_degdist.pkl','wb'))
pkl.dump(meanconn, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_meanconn.pkl','wb'))

# dat TRAIN demographic info
pkl.dump(age, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_ages.pkl', 'wb')) # dict of subjids: ages
pkl.dump(behav, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_behav.pkl', 'wb')) # dict of subjids: {demog/cog: data}
pkl.dump(subjids, open('/Users/nalsharif/code/nooralsh/ping_proc/data/train_subjids.pkl','wb')) # list of subjids
